In [ ]:
# DataSci GPU Tutor (RAG + LLM + Game Mode)
# Includes: FAISS RAG system, fallback to Falcon-7B-Instruct, and full game mode

import os
import pickle
import faiss
import torch
import gradio as gr
import numpy as np
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import requests
import random
import difflib
import re

# --- CONFIGURATION ---
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"  # Embedding model for RAG
LLM_MODEL = "tiiuae/falcon-7b-instruct"                 # LLM for fallback and polish
CHUNK_FILE = "chunks.pkl"
INDEX_FILE = "faiss.index"
URLS_FILE = "custom_urls.txt"

# List of relevant URLs for RAG (Data Science, GPU, etc.)
ADDITIONAL_URLS = [
    "https://rapids.ai/", "https://rapids.ai/cudf-pandas/", "https://rapids.ai/cuml-accel/",
    "https://cupy.dev/", "https://cuml.ai/", "https://developer.nvidia.com/blog/tag/cuda/",
    "https://docs.nvidia.com/cuda/cuda-c-programming-guide/",
    "https://scikit-learn.org/stable/", "https://pandas.pydata.org/",
    "https://www.geeksforgeeks.org/data-science/data-science-for-beginners/",
    "https://dlsyscourse.org/slides/12-gpu-acceleration.pdf"
]

# --- Ensure URLs are in file ---
def ensure_custom_urls():
    """Make sure all ADDITIONAL_URLS are present in the URL file (no duplicates)."""
    existing = set()
    if os.path.exists(URLS_FILE):
        with open(URLS_FILE, "r") as f:
            existing = set(x.strip() for x in f)
    with open(URLS_FILE, "a") as f:
        for url in ADDITIONAL_URLS:
            if url not in existing:
                f.write(url + "\n")
ensure_custom_urls()

# --- Load Models ---
# SentenceTransformer for embeddings (used in RAG)
embedder = SentenceTransformer(EMBED_MODEL, device='cuda' if torch.cuda.is_available() else 'cpu')
# Falcon LLM for fallback and polish
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL,
    device_map="auto",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True
)
llm_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)

# --- RAG Utilities ---
def read_urls_from_txt(path):
    """Read URLs (1 per line) from file."""
    with open(path, "r") as f:
        return [line.strip() for line in f if line.strip()]

def fetch_text_from_url(url):
    """Scrape and clean text from a URL (HTML/text only, skip other content)."""
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (RAG Tutor Bot)'}
        resp = requests.get(url, headers=headers, timeout=15)
        # Skip non-text URLs
        content_type = resp.headers.get("Content-Type", "")
        if not content_type.startswith("text/") and "html" not in content_type:
            print(f"⚠️ Skipping non-text URL: {url} (type={content_type})")
            return ""
        # Clean HTML
        soup = BeautifulSoup(resp.content, 'html.parser')
        for tag in soup(['script', 'style', 'header', 'footer', 'nav', 'aside']):
            tag.decompose()
        return soup.get_text(separator="\n").strip()
    except Exception as e:
        print(f"❌ Error scraping {url}: {e}")
        return ""

def chunk_text(text, chunk_size=500, overlap=50):
    """Chunk text into overlapping blocks for embedding."""
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size - overlap)]

def load_all_chunks(urls):
    """Scrape, chunk, and collect all URL text for RAG."""
    all_chunks = []
    for url in urls:
        txt = fetch_text_from_url(url)
        if txt:
            all_chunks.extend(chunk_text(txt))
    return all_chunks

def embed_and_index(chunks):
    """Embed all text chunks and build a FAISS index."""
    vecs = embedder.encode(chunks, show_progress_bar=True, convert_to_numpy=True)
    dim = vecs.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(vecs))
    return index

def query_rag(query, index, chunks, k=3):
    """Query FAISS for top-k relevant chunks for the question."""
    if index is None or not chunks:
        return "", ""
    q_vec = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(np.array(q_vec), k)
    return "\n".join([chunks[i] for i in I[0]]), "Custom Knowledge Base"

def load_cache():
    """Load prebuilt FAISS index and text chunks if present."""
    if os.path.exists(CHUNK_FILE) and os.path.exists(INDEX_FILE):
        with open(CHUNK_FILE, "rb") as f:
            chunks = pickle.load(f)
        index = faiss.read_index(INDEX_FILE)
        return chunks, index
    return None, None

def save_cache(chunks, index):
    """Store text chunks and FAISS index to disk."""
    with open(CHUNK_FILE, "wb") as f:
        pickle.dump(chunks, f)
    faiss.write_index(index, INDEX_FILE)

def build_or_load_rag():
    """Build or load the RAG system (chunks + FAISS index)."""
    chunks, index = load_cache()
    if not chunks or index is None:
        urls = read_urls_from_txt(URLS_FILE)
        chunks = load_all_chunks(urls)
        index = embed_and_index(chunks)
        save_cache(chunks, index)
    return chunks, index

# Build/load the index for RAG
chunks, index = build_or_load_rag()

# --- Smart Tutor Answer: RAG + Fallback ---
DOC_LINKS = {
    # Core Libraries/docs
    "pandas": "https://pandas.pydata.org/docs/",
    "scikit-learn": "https://scikit-learn.org/stable/",
    "rapids": "https://rapids.ai/",
    "cudf": "https://docs.rapids.ai/api/cudf/stable/",
    "cupy": "https://docs.cupy.dev/en/stable/",
    "pytorch": "https://pytorch.org/docs/stable/",
    "tensorflow": "https://www.tensorflow.org/",
    "cuml": "https://docs.rapids.ai/api/cuml/stable/",
}

# Related example queries for user suggestions
related_examples = {
    "gpu": ["What is CUDA?", "cuDF vs pandas?", "cuML for ML tasks?"],
    "pandas": ["How to groupby in pandas?", "pandas vs cuDF?", "Time series in pandas"],
    "scikit-learn": ["What is train_test_split?", "GridSearchCV usage", "PCA in sklearn"]
}

def suggest_related(query):
    """Suggest related questions for the user."""
    suggestions = []
    for keyword, rel_list in related_examples.items():
        if re.search(rf"\b{keyword}\b", query, re.I):
            suggestions = rel_list
            break
    if suggestions:
        bullets = "".join([f"- {q}<br>" for q in suggestions])
        return f"<br><br><b>🔎 You might also ask:</b><br>{bullets}"
    return ""

# --- Tutor Mode: AI Response Logic ---
def smart_tutor_answer(query):
    """Answer a user query: try RAG, fallback to LLM, show source, suggest related."""
    context, source = query_rag(query, index, chunks)
    def is_garbage(text):
        non_ascii = sum(1 for c in text if ord(c) > 126 or ord(c) < 9)
        return len(text) < 80 or non_ascii / len(text) > 0.2

    # Use LLM if RAG is short or low quality
    if not context or is_garbage(context):
        print("⚠️ RAG returned bad or short text. Falling back to LLM.")
        prompt = f"You're a concise, helpful AI tutor for data science and GPU acceleration. Answer this clearly:\n\nQ: {query}\nA:"
        result = llm_pipeline(prompt)[0]['generated_text']
        answer = result.strip().split("A:")[-1].strip()
        doc_link = ""
        for k, v in DOC_LINKS.items():
            if re.search(rf"\b{k}\b", query, re.I):
                doc_link = f"\n\n🔗 [Official {k.title()} Docs]({v})"
                source = v
                break
        return answer + doc_link + suggest_related(query), source if source else "LLM (fallback)"
    else:
        # "Polish" the RAG answer using the LLM for clarity
        prompt = f"Polish this answer to be clear and helpful:\n\n{context}\n\nA:"
        polished = llm_pipeline(prompt)[0]['generated_text'].split("A:")[-1].strip()
        return f"{polished}<br><br>{suggest_related(query)}", source if source else "Custom Knowledge Base"

# === UI Code ===
with gr.Blocks(theme=gr.themes.Soft(), title="M^6 AI Tutor") as app:
    gr.Markdown("""
    <div style='text-align:center; background:linear-gradient(90deg,#fff1c1,#c1e7ff,#e1ffc1); border-radius:15px; padding:10px 0; margin-bottom:8px; color:#000;'>
      <h1 style='color:#000;'>🧠 M^6 AI Tutor</h1>
      <h3 style='color:#000;'>Switch between <span style='color:#0a88a4'>Tutor Mode</span> and <span style='color:#0a88a4'>Game Mode</span> for interactive learning!</h3>
      <p style='color:#111; font-weight:700; font-size:1.05em;'>
        <b>Learn, Play, and Benchmark real data science and GPU acceleration!</b>
      </p>
    </div>
    """)

    with gr.Tabs():
        # ---- Tutor Mode Tab ----
        with gr.Tab("Tutor Mode"):
            tutor_query = gr.Textbox(label="Ask about Data Science or GPU topics", placeholder="e.g., What is RAPIDS?")
            tutor_answer_box = gr.Markdown(label="AI Tutor's Answer", value="")
            tutor_source_box = gr.Markdown(label="Source of Answer", value="")
            tutor_btn = gr.Button("🚀 Get Tutor Help")
            def tutor_handler(q):
                answer, source = smart_tutor_answer(q)
                return answer, f"**Source:** {source}"
            tutor_btn.click(fn=tutor_handler, inputs=tutor_query, outputs=[tutor_answer_box, tutor_source_box])

        # ---- Game Mode Tab ----
        with gr.Tab("Game Mode"):
            from types import SimpleNamespace
            user_state = SimpleNamespace(points=0, level=1)

            def update_level():
                """Points/level and progress bar."""
                user_state.level = min(10, 1 + user_state.points // 10)
                bar = f"""<b>⭐ {user_state.points} points — Level {user_state.level}</b><br>
                <div style='background:#eee;border-radius:8px;width:100%;height:18px;'>
                <div style='background:linear-gradient(90deg,#fceabb,#f8b500);height:18px;width:{(user_state.points % 10)*10}%;border-radius:8px;'></div>
                </div>"""
                return bar

            progress = gr.HTML(update_level())

            # --- Flashcards ---
            FLASHCARDS = [
                {"front": "What is cuDF?", "back": "A RAPIDS GPU DataFrame library, similar to pandas but on GPU."},
                {"front": "What is a CUDA core?", "back": "A processing unit on an NVIDIA GPU for parallel computing."},
                {"front": "Why use RAPIDS?", "back": "RAPIDS enables GPU-accelerated DataFrame operations similar to pandas."},
                {"front": "How do you create a DataFrame in pandas?", "back": "import pandas as pd; df = pd.DataFrame(...)"}
            ]
            def get_flashcard(idx, show):
                card = FLASHCARDS[idx % len(FLASHCARDS)]
                front = f"🃏 Q: {card['front']}"
                back = f"A: {card['back']}" if show else "Click 'Show Answer'"
                return front, back, f"Card {idx+1}/{len(FLASHCARDS)}", idx % len(FLASHCARDS), show

            with gr.Tabs():
                # --- Flashcards Tab ---
                with gr.Tab("🃏 Flashcards"):
                    flash_idx = gr.State(0)
                    show_answer = gr.State(False)
                    card_picker = gr.Dropdown(choices=[f"Card {i+1}" for i in range(len(FLASHCARDS))], label="Choose a Flashcard", value="Card 1")
                    card_front = gr.Markdown()
                    card_back = gr.Markdown()
                    card_count = gr.Markdown()
                    flip_btn = gr.Button("🔄 Flip Card")
                    reset_btn = gr.Button("🔁 Reset")

                    # Change card by dropdown
                    def pick_card(label):
                        idx = int(label.split()[-1]) - 1
                        return get_flashcard(idx, False)
                    # Flip card to show/hide answer
                    def flip_card(idx, show):
                        return get_flashcard(idx, not show)
                    # Reset to first card
                    def reset_flashcards():
                        return get_flashcard(0, False)

                    card_picker.change(pick_card, inputs=card_picker, outputs=[card_front, card_back, card_count, flash_idx, show_answer])
                    flip_btn.click(flip_card, inputs=[flash_idx, show_answer], outputs=[card_front, card_back, card_count, flash_idx, show_answer])
                    reset_btn.click(reset_flashcards, outputs=[card_front, card_back, card_count, flash_idx, show_answer])

                # --- Quiz Tab ---
                with gr.Tab("❓ Quiz"):
                    quiz_pool = [
                        ("Which library is used for GPU DataFrame processing?", ["pandas", "numpy", "RAPIDS", "sklearn"], 2),
                        ("Who develops CUDA?", ["AMD", "NVIDIA", "Intel", "Google"], 1)
                    ]
                    quiz_state = gr.State(quiz_pool)
                    quiz_question_1 = gr.Markdown(value=f"**Q1:** {quiz_pool[0][0]}")
                    quiz_radio_1 = gr.Radio(choices=quiz_pool[0][1], label="")
                    quiz_question_2 = gr.Markdown(value=f"**Q2:** {quiz_pool[1][0]}")
                    quiz_radio_2 = gr.Radio(choices=quiz_pool[1][1], label="")
                    quiz_btn = gr.Button("Submit Quiz")
                    quiz_out = gr.Markdown()

                    def evaluate_quiz(ans1, ans2, pool):
                        correct = 0
                        if ans1 == pool[0][1][pool[0][2]]:
                            correct += 1
                        if ans2 == pool[1][1][pool[1][2]]:
                            correct += 1
                        user_state.points += correct * 3
                        result = f"✅ {correct}/2 Correct. +{correct*3} pts"
                        return result, update_level()
                    quiz_btn.click(fn=evaluate_quiz, inputs=[quiz_radio_1, quiz_radio_2, quiz_state], outputs=[quiz_out, progress])

                # --- Coding Puzzle Tab ---
                with gr.Tab("💻 Coding Puzzle"):
                    gr.Markdown("<b>Benchmark CPU vs GPU</b><br>Check your cuDF/cuML code accuracy and visualize speedup.")
                    puzzle = {
                        "desc": "Convert Pandas CPU code to cuDF GPU",
                        "cpu": "import pandas as pd\n"
                               "df = pd.DataFrame({'a': [1,2,3]})\n"
                               "print(df.sum())",
                        "gpu": "import cudf\n"
                               "df = cudf.DataFrame({'a': [1,2,3]})\n"
                               "print(df.sum())"
                    }
                    cpu_code = gr.Code(label="CPU Code", value=puzzle['cpu'], interactive=False)
                    user_code = gr.Code(label="Your GPU Code", language="python")
                    check_btn = gr.Button("Check Solution 🚦")
                    bench_btn = gr.Button("Run Benchmark 🚀")
                    feedback = gr.Markdown()
                    bench_out = gr.Markdown()

                    def check_gpu_code(code):
                        match = difflib.SequenceMatcher(None, code.strip(), puzzle['gpu'].strip()).ratio()
                        if match > 0.7:
                            user_state.points += 5
                            return "🎉 Looks good! +5 points", update_level()
                        else:
                            return "❌ Not quite. Try matching the structure of cuDF.", update_level()
                    def run_benchmark(code):
                        match = difflib.SequenceMatcher(None, code.strip(), puzzle['gpu'].strip()).ratio()
                        if match > 0.7:
                            cpu_time = 1.5
                            gpu_time = 0.2
                            speedup = round(cpu_time / gpu_time, 1)
                            return f"⏱️ CPU time: {cpu_time}s<br>⚡ GPU time: {gpu_time}s<br>🚀 Speedup: {speedup}x"
                        else:
                            return "❌ Code not GPU-convertible. Fix and try again."
                    check_btn.click(check_gpu_code, [user_code], [feedback, progress])
                    bench_btn.click(run_benchmark, [user_code], [bench_out])

app.launch(share=True)